In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

# !pip install transformers torch librosa numpy pandas matplotlib


In [3]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 MB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.8/319.8 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 436.6/436.6 kB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 63.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/1

In [1]:
import os
import numpy as np
import librosa
import torch
import matplotlib.pyplot as plt
from transformers import Wav2Vec2ForCTC, Wav2Vec2Tokenizer, DistilBertTokenizer, DistilBertForSequenceClassification
import gradio as gr
import logging
import traceback


In [2]:
# Cell 2: Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


In [3]:
# Cell 3: Load models (ensure these are the fine-tuned versions)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
logger.info(f"Using device: {device}")

try:
    wav2vec2_tokenizer = Wav2Vec2Tokenizer.from_pretrained("facebook/wav2vec2-base-960h")
    wav2vec2_model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h").to(device)
    distilbert_tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased-finetuned-sst-2-english')
    distilbert_model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased-finetuned-sst-2-english').to(device)
    logger.info("Models loaded successfully")
except Exception as e:
    logger.error(f"Error loading models: {str(e)}")
    raise


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:90: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.60k [00:00<?, ?B/s]

...

Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.masked_spec_embed']
You sho

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [4]:
# Cell 4: Define audio processing functions
def speech_to_text(audio):
    try:
        input_values = wav2vec2_tokenizer(audio, return_tensors="pt").input_values.to(device)
        logits = wav2vec2_model(input_values).logits
        predicted_ids = torch.argmax(logits, dim=-1)
        transcription = wav2vec2_tokenizer.batch_decode(predicted_ids)[0]
        return transcription
    except Exception as e:
        logger.error(f"Error in speech_to_text: {str(e)}")
        return f"Error in speech_to_text: {str(e)}\n{traceback.format_exc()}"

def analyze_sentiment(text):
    try:
        inputs = distilbert_tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(device)
        outputs = distilbert_model(**inputs)
        sentiment = torch.argmax(outputs.logits, dim=1)
        sentiment_label = "Positive" if sentiment.item() == 1 else "Negative"
        return sentiment_label
    except Exception as e:
        logger.error(f"Error in analyze_sentiment: {str(e)}")
        return f"Error in analyze_sentiment: {str(e)}\n{traceback.format_exc()}"

def extract_audio_features(audio):
    try:
        # Calculate pitches
        pitches, _ = librosa.piptrack(y=audio, sr=22050)
        pitch_values = pitches[pitches > 0]  # Get non-zero pitch values
        pitch = np.mean(pitch_values) if pitch_values.size > 0 else 0  # Calculate mean pitch

        # Calculate tempo
        tempo, _ = librosa.beat.beat_track(y=audio, sr=22050)

        # Calculate spectral centroid (tone)
        spectral_centroids = librosa.feature.spectral_centroid(y=audio, sr=22050)
        tone = np.mean(spectral_centroids) if spectral_centroids.size > 0 else 0

        return float(pitch), float(tempo), float(tone)  # Ensure these are floats
    except Exception as e:
        logger.error(f"Error in extract_audio_features: {str(e)}")
        return 0.0, 0.0, 0.0  # Return zeros as floats on error




def process_audio(audio, sample_rate):
    try:
        logger.info(f"Processing audio with sample rate: {sample_rate}")

        # Convert audio to float32 if it's not already
        if audio.dtype != np.float32:
            audio = audio.astype(np.float32)

        # Normalize audio to be in the range [-1.0, 1.0]
        audio = audio / np.max(np.abs(audio))

        # Resample audio to 16kHz for Wav2Vec2
        audio_16k = librosa.resample(y=audio, orig_sr=sample_rate, target_sr=16000)

        # Speech to text
        transcription = speech_to_text(audio_16k)
        logger.info(f"Transcription: {transcription}")

        # Sentiment analysis
        sentiment = analyze_sentiment(transcription)
        logger.info(f"Sentiment: {sentiment}")

        # Extract audio features
        pitch, tempo, tone = extract_audio_features(audio_16k)
        logger.info(f"Audio features - Pitch: {pitch}, Tempo: {tempo}, Tone: {tone}")

        return transcription, sentiment, pitch, tempo, tone
    except Exception as e:
        logger.error(f"Error in process_audio: {str(e)}")
        return f"Error in process_audio: {str(e)}\n{traceback.format_exc()}", "Error", 0, 0, 0


In [5]:
# Cell 5: Define visualization functions
def plot_waveform(audio):
    plt.figure(figsize=(10, 4))
    plt.plot(audio)
    plt.title('Audio Waveform')
    plt.xlabel('Sample')
    plt.ylabel('Amplitude')
    plt.grid()
    plt.close()
    return plt

def plot_features(pitch, tempo, tone):
    labels = ['Pitch', 'Tempo', 'Tone']
    values = [pitch, tempo, tone]

    # Ensure values are all floats for the bar plot
    logger.info(f"Plotting features with values: {values} (Types: {[type(v) for v in values]})")

    plt.figure(figsize=(10, 4))
    plt.bar(labels, values, color=['blue', 'orange', 'green'])
    plt.title('Audio Features')
    plt.ylabel('Value')
    plt.ylim(0, max(values) + 1)  # Set appropriate y-limits
    plt.close()
    return plt




In [6]:
def audio_sentiment_analyzer(audio):
    if audio is None:
        logger.warning("No audio received")
        return "No audio recorded", "N/A", None, None

    try:
        sr, audio = audio
        logger.info(f"Received audio with sample rate: {sr} and shape: {audio.shape}")

        transcription, sentiment, pitch, tempo, tone = process_audio(audio, sr)

        # Debugging: Log pitch, tempo, tone types and values
        logger.info(f"Pitch: {pitch} (Type: {type(pitch)})")
        logger.info(f"Tempo: {tempo} (Type: {type(tempo)})")
        logger.info(f"Tone: {tone} (Type: {type(tone)})")

        # Create visualizations
        fig_waveform = plot_waveform(audio)
        fig_features = plot_features(pitch, tempo, tone)

        return transcription, sentiment, fig_waveform, fig_features
    except Exception as e:
        error_msg = f"Error in audio_sentiment_analyzer: {str(e)}\n{traceback.format_exc()}"
        logger.error(error_msg)
        return error_msg, error_msg, None, None


In [7]:
# Cell 7: Create Gradio interface
iface = gr.Interface(
    fn=audio_sentiment_analyzer,
    inputs=gr.Audio(type="numpy"),  # Adjusted the input here
    outputs=[
        gr.Textbox(label="Transcription"),
        gr.Textbox(label="Sentiment"),
        gr.Plot(label="Audio Waveform"),
        gr.Plot(label="Audio Features")
    ],
    title="Audio Sentiment Classifier",
    description="Record audio to analyze its sentiment and transcribe the speech.",
    theme="huggingface",
    allow_flagging="never"
)


/usr/local/lib/python3.10/dist-packages/gradio/blocks.py:1027: UserWarning: Cannot load huggingface. Caught Exception: 404 Client Error: Not Found for url: https://huggingface.co/api/spaces/huggingface (Request ID: Root=1-67079343-1160ca9726102e733bae1d23;cb9057f9-8786-4ee6-890b-6b5071061ce9)

Sorry, we can't find the page you are looking for.
  warnings.warn(f"Cannot load {theme}. Caught Exception: {str(e)}")
/usr/local/lib/python3.10/dist-packages/gradio/interface.py:393: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


In [8]:
# Cell 8: Launch Gradio interface
iface.launch(debug=True, share=True)


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://3f05d635ec8c8be9fe.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


<ipython-input-4-21d5ee66f54a>:38: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return float(pitch), float(tempo), float(tone)  # Ensure these are floats


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://3f05d635ec8c8be9fe.gradio.live


In [9]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [10]:
%cd /content/drive/MyDrive/ai_app_development


/content/drive/MyDrive/ai_app_development


In [11]:
!git init

Reinitialized existing Git repository in /content/drive/MyDrive/ai_app_development/.git/


In [13]:
!git add .
!git commit -m "Initial commit with project files"


Author identity unknown

*** Please tell me who you are.

Run

  git config --global user.email "you@example.com"
  git config --global user.name "Your Name"

to set your account's default identity.
Omit --global to set the identity only in this repository.

fatal: unable to auto-detect email address (got 'root@988c69a8ed7b.(none)')
